# The aHB circuit tracks fictive trajectories

We have seen in the previous notebooks that the aHB integrator could be capabable of keep track of past motion over a significant amount of time. In this notebook, we will see how much it can keep track of past trajectories over significant amounts of time.

## A first look at cumulative directional motion and network phase

In [ ]:
%matplotlib widget
from pathlib import Path

import lotr.plotting as pltltr
import numpy as np
import pandas as pd
from bouter.utilities import crop
from lotr import A_FISH, LotrExperiment, dataset_folders
from lotr.behavior import get_bouts_props_array
from lotr.default_vals import REGRESSOR_TAU_S
from lotr.utils import convolve_with_tau, linear_regression, nan_phase_jumps
from matplotlib import pyplot as plt
from scipy.stats import pearsonr, spearmanr, wilcoxon
from tqdm import tqdm

from lotr.result_logging import ResultsLogger
logger = ResultsLogger()

COLS = pltltr.COLS

In [ ]:
# First, get the unwrapped phase:
exp = LotrExperiment(A_FISH)
network_phase = np.unwrap(exp.network_phase)

# Now, let's find a way of reconstruct past motion of the fish.
# We will start with a vector of zeros that has ones at timepoints
# where the fish swam:
theta_turned = get_bouts_props_array(
    exp.n_pts, exp.bouts_df, selection="all", value="bias",
)

# Fictive heading will be the cumulative sum of this array:
fictive_head = np.cumsum(theta_turned)

# Finally, we smooth this array with a kernel that matches the slower dynamics
# of the neuron (results are not much affected by this)
fictive_head = convolve_with_tau(np.cumsum(theta_turned), REGRESSOR_TAU_S * exp.fn)

In [ ]:
f, axs = plt.subplots(
    3, 1, figsize=(6., 3), sharex=True, gridspec_kw=(dict(bottom=0.12))
)
lw = 1

labels = ["Tail angle", "Angle turned (dθ)", "Fict. heading (θ)"]
col_seq = [COLS["beh"], pltltr.shift_lum(COLS["th_plot"], 0.4), COLS["th_plot"]]
for i, (x, y) in enumerate(
    [
        (exp.behavior_log["t"], exp.behavior_log["tail_sum"]),
        (exp.time_arr, theta_turned),
        (exp.time_arr, fictive_head),
    ]
):
    axs[i].plot(
        x, y, lw=lw, c=col_seq[i], label=labels[i], rasterized=i == 0
    )  # rasterize for exporting
    axs[i].legend(loc=2, bbox_to_anchor=(0.8, 0.95), labelcolor="linecolor",
        handlelength=0.0)

axs[0].set(**pltltr.get_pi_labels(1, ax="y"))
axs[1].set(**pltltr.get_pi_labels(0.5, ax="y"), ylim=(-1.8, 1.8))
axs[2].set(xlabel="Time (s)", **pltltr.get_pi_labels(coefs=(0, -5, -10, -15), ax="y"))

[pltltr.despine(ax) for ax in axs.flatten()]
plt.show()
pltltr.savefig("fictive_traj_computation", folder="S7")

Now, how does the network phase look, compared with the fictive heading?

In [ ]:
t_off_s, t_dur_s = 150, 400

f, ax = plt.subplots(
    1, 1, figsize=(4, 1.5), gridspec_kw=(dict(bottom=0.25, right=0.85))
)
twin_ax = ax.twinx()
ax.plot(
    exp.time_arr, fictive_head, c=COLS["th_plot"],
)
twin_ax.plot(
    exp.time_arr, np.unwrap(exp.network_phase), c=COLS["ph_plot"],
)
ax.set_ylabel("Fict. heading (θ)", c=COLS["th_plot"])
ax.set(xlabel="Times (s)", **pltltr.get_pi_labels(coefs=(0, -5, -10, -15, -20), ax="y"))
twin_ax.set_ylabel("Network phase (ϕ)", c=COLS["ph_plot"])
twin_ax.set(**pltltr.get_pi_labels(coefs=(0, 2, 4, 6), ax="y"))
ax.set(ylim=(2, -65))
# rho, pval = spearmanr(fictive_head, np.unwrap(exp.network_phase))
# ax.text(1200, -np.pi, "$ρ_r=" + f"{rho:0.3f}" + "$" + pltltr.get_pval_stars(pval))
[pltltr.despine(a, ["top"]) for a in [ax, twin_ax]]
ax.axvspan(t_off_s, t_off_s + t_dur_s, lw=0, fc=".9", zorder=-100)
pltltr.savefig("fictive_traj_phase_onefish")

## Quantify correlation over the entire dataset

To run the correlation over the entire dataset, we will not use the whole trace to avoid the problem that a large drift at some point of the experiment could disrupt the correlation for the rest of it. Instead, we will use correlation in windows of 5 minutes, including only times where there were at least 2 directional bouts happening (otherwise fictive heading will be a flat line, with which correlation ill-defined).

In [ ]:
np.random.seed(34897010)

CORR_WND_S = 300  # window over which correlation will be computed
N_OVERLAPS = 10  # number of overlaps per window
MIN_BOUTS = 2  # minimum number of bouts to inclide in the window

results_df = []
for path in tqdm(dataset_folders):
    # for path in tqdm(maybe):
    exp = LotrExperiment(path)
    phase = np.unwrap(exp.network_phase)
    theta_turned = get_bouts_props_array(
        exp.n_pts, exp.bouts_df, selection="all", value="bias",
    )

    # Fictive heading will be the cumulative sum of this array:
    fictive_head = np.cumsum(theta_turned)
    wnd_pts = int(CORR_WND_S * exp.fn)
    overlap_pts = wnd_pts // N_OVERLAPS

    # Find what ranges are valid, i.e have enough bouts in the interval:
    indexes = list(range(0, len(phase) - wnd_pts, overlap_pts))
    valid_indexes = []
    for i in indexes:
        if np.sum(np.abs(theta_turned[i : i + wnd_pts])) > MIN_BOUTS:
            valid_indexes.append(i)

    shuf_indexes = []
    for i in valid_indexes:
        shuf_i = i
        c = 0
        while np.abs(i - shuf_i) < wnd_pts // 2:
            shuf_i = np.random.choice(valid_indexes)
            c += 1
            if c > 50:
                print("could not find a valid shuffle for ", path.name)
                break

        shuf_indexes.append(shuf_i)

    for i, shuf_i in zip(valid_indexes, shuf_indexes):
        wnd = slice(i, i + wnd_pts)
        shuf_wnd = slice(shuf_i, shuf_i + wnd_pts)

        toshuf = fictive_head[shuf_wnd]

        if np.random.rand() > 0.5:
            toshuf = -toshuf

        r_corr, _ = pearsonr(
            phase[wnd], fictive_head[wnd]
        )  # spearmanr(phase[wnd], fictive_head[wnd])
        r_shuf, _ = pearsonr(phase[wnd], toshuf)  # spearmanr(phase[wnd], toshuf)

        results_df.append(dict(r_corr=r_corr, shuf=r_shuf, fid=exp.dir_name))

In [ ]:
res_df = pd.DataFrame(results_df)
# compute quantiles:
quantiles_df = res_df.groupby("fid").quantile((0.25, 0.50, 0.75))

# sort on 0.5 quantile:
sort_order = quantiles_df.xs(0.5, level=1).sort_values("r_corr").index

# compute p values for each fish:
def _wilcoxon_p(df, keys=["r_corr", "shuf"]):
    _, p = wilcoxon(*[df[k] for k in keys])
    return p


pvals_df = res_df.groupby("fid").apply(_wilcoxon_p)

In [ ]:
f, ax = plt.subplots(figsize=(3, 2), gridspec_kw=dict(bottom=0.2, left=0.18),)
for k, lab, col in zip(
    ["shuf", "r_corr"], ["Shuffle", "Data"], [".5", COLS["ph_plot"]]
):
    stacked_quantiles = quantiles_df[[k]].stack().unstack(level=0)
    pltltr.tick_with_bars(
        stacked_quantiles[sort_order], cols=[col,] * len(sort_order), s=0.2,
    )
    # Dummy plot for legend:
    plt.plot([np.nan], [np.nan], c=col, label=lab)


ax.legend(loc=4, fontsize=7, labelcolor="linecolor",
        handlelength=0.0)
# plt.plot(rhos[np.argsort(rhos[:, 1]), :], c="r")
ax.set(xlabel="Fish (sorted by corr.)", ylabel="Heading/phase corr.")
ax.axhline(0, lw=1, c=".4")
pltltr.despine(ax)
plt.show()

pltltr.savefig("correlations_all_fish_nostats")
y_txt = 0.7
for i, pval in enumerate(pvals_df[sort_order]):
    plt.text(
        i,
        y_txt,
        pltltr.get_pval_stars(pval),
        fontsize=6,
        rotation="vertical",
        zorder=100,
        va="top",
    )

pltltr.savefig("correlations_all_fish_stats")

In [ ]:
f, ax = plt.subplots(figsize=(1.5, 2.5), gridspec_kw=(dict(bottom=0.2, left=0.4)))
ax.scatter(np.ones(len(pvals_df)) + np.random.rand(len(pvals_df))*0.4, -np.log10(pvals_df), s=10)
ax.set(xlim=(0, 2), ylabel="-log10(p_val)", ylim=(0.5, 11))
ax.axhline(-np.log10(0.05), lw=0.5, c=".6")
pltltr.despine(ax, ["bottom", "right", "top"])
pltltr.savefig("pvals_distr", folder="S7")

In [ ]:
stacked_quantiles = quantiles_df[["r_corr"]].stack().unstack(level=0)
logger.add_entry("heading_phase_corr", stacked_quantiles.xs((0.5, "r_corr")), stacked_quantiles.xs((0.5, "r_corr")).index, moment="median")


In [ ]:
logger.add_entry("pvals_corrs", pvals_df, stacked_quantiles.xs((0.5, "r_corr")).index, moment="minmax")

### Traces for all fish

In [ ]:
f, axs = plt.subplots(11, 3, figsize=(8, 8), gridspec_kw=dict(wspace=0.35, hspace=0.4))

axs_flat = axs.flatten()
results_df = []
for i, path in tqdm(enumerate(dataset_folders)):
    # for path in tqdm(maybe):
    exp = LotrExperiment(path)
    phase = np.unwrap(exp.network_phase)
    theta_turned = get_bouts_props_array(
        exp.n_pts, exp.bouts_df, selection="all", value="bias",
    )
    fictive_head = np.cumsum(theta_turned)

    ax = axs_flat[i]
    
    twin_ax = ax.twinx()
    ax.plot(
        exp.time_arr, fictive_head, c=COLS["th_plot"], label="Estimated heading"
    )
    ax.plot([], [], c=COLS["ph_plot"], label="Network phase"
    )
    twin_ax.plot(
        exp.time_arr, np.unwrap(exp.network_phase), c=COLS["ph_plot"],
    )
    ax.set_title(path.name, fontsize=6, ha="right")
    lims = twin_ax.get_ylim()
    twin_ax.set_ylim((lims[1], lims[0]))
    pltltr.add_scalebar(ax, xlen=500, ylen=0, disable_axis=False, xlabel="", ylabel="")

    [pltltr.despine(a, ["top", "bottom"]) for a in [ax, twin_ax]]
    for a, col in zip([ax, twin_ax], [COLS["th_plot"], COLS["ph_plot"]]):
        [t.set_color(col) for t in a.yaxis.get_ticklines()]
        [t.set_color(col) for t in a.yaxis.get_ticklabels()]
# ax.axvspan(t_off_s, t_off_s + t_dur_s, lw=0, fc=".9", zorder=-100)

[pltltr.despine(axs_flat[j], "all") for j in range(i+1, len(axs_flat))]
    

axs[0, -1].legend(loc=2, bbox_to_anchor=(0.55, 2), labelcolor="linecolor",
        handlelength=0.0)

pltltr.savefig("phase_heading_allfish", folder="S7")

In [ ]:
axs[0, -1].legend(loc=2, bbox_to_anchor=(0.4, 2), labelcolor="linecolor", ha="right",
        handlelength=0.0)